In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
BASE_DIR = os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir))
sys.path.append(BASE_DIR)

In [ ]:
import os
import time
import imgaug.augmenters as iaa
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from models.dcgan import DCGAN

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')    
tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])

In [ ]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [ ]:
print(train_images.shape)

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)

In [ ]:
aug = iaa.Resize({"height": 64, "width": 64})

In [ ]:
show_batch(train_images[:36])

In [ ]:
show_batch(aug(images=train_images[:36]))

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
train_images = aug(images=train_images)
train_images = (train_images.astype(np.float32) - 127.5) / 127.5 # 将图片标准化到 [-1, 1] 区间内

In [ ]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
noise_dim = 100

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
dcgan = DCGAN(image_shape=(64, 64, 1))

In [ ]:
def show_batch(images):
    fig = plt.figure(figsize=(6, 6))
    for i in range(images.shape[0]):
        plt.subplot(6, 6, i+1)
        image = images[i, :, :, 0] * 127.5 + 127.5
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
images = next(iter(train_dataset))
show_batch(images[: 36])

In [ ]:
num_examples_to_generate = 36

seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [ ]:
def show_generator(generator, seed):
    predictions = generator(seed, training=False)
    show_batch(predictions)

In [ ]:
show_generator(dcgan.generator, seed)

In [ ]:
for _ in range(100):
    for image_batch in train_dataset:
        dcgan.train_step(image_batch, num_iter_disc=1, num_iter_gen=1)
    show_generator(dcgan.generator, seed)